# Proefening 3

In deze tweede oefening gaan we een complexere analyse doen, door te kijken welke punten overlappen met een deel van de BGT vlakken.
Run hiervoor eerst het notebook download_bgt.ipynb. Hier kan je eerst de beschikbare collecties van de BGT printen, en vervolgens een deel van de BGT downloaden.
Deze wordt automatisch in je output folder opgeslagen.

het script maakt gebruik van de bounding box uit de env.yml file. De bounding box is in het format [minx, miny, maxx, maxx]. Uit je AHN5 file kan je de minimale waardes al halen. Tel hier 1000 bij op (want het is 1km bij 1km) voor de max waardes. Pas deze waarde voor nu niet aan, omdat die overeenkomt met je data.

In de env.yml staan deze al goed, als je geen eigen .las file hebt gekozen!

De output wordt opgeslagen in je output folder, als bgt_collections.gpkg.

### Package imports

In [95]:
import laspy as lp
import numpy as np
import geopandas as gpd
import open3d as o3d
import os
import yaml
import logging
from typing import Optional
from pathlib import Path

In [96]:
### Load environment variables
def load_yaml(path_to_yaml: str):
    """
    Load yaml file
    """
    with open(path_to_yaml, "r") as file:
        return yaml.safe_load(file)


env = load_yaml("env.yml")
data_folder = env["data_folder"]
output_folder = Path(data_folder) / "output"
bbox = env["bbox"]
bgt_name = env["bgt_name"]
subset = env["subset"]

### Handige functies

Onderstaande functies helpen ons in deze workshop. Bij elke functie zie je een korte beschrijving van wat hij doet.

In [97]:
def get_files_in_directory(directory: str):
    """
    Get all files with a specific extension in a directory.
    """

    files = [
        os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".las")
    ]
    return files


def lasdata_loader(filepaths: str | list, single_array: bool = True):
    """
    Load one or multiple .las or .laz files.
    Can return the data as a single numpy array (if single_array=True)
    or as a list of numpy arrays (if single_array=False).
    """
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    all_points = []
    for filepath in filepaths:
        las = lp.read(filepath)
        coords = np.vstack((las.points.x, las.points.y, las.points.z))
        point_cloud = coords.transpose()
        all_points.append(point_cloud)

    if single_array:
        all_points = np.concatenate(all_points)

    return all_points


def to_las(
    points: np.ndarray, colors: Optional[np.ndarray], extra_dim: Optional[dict]
) -> lp.LasData | None:
    """
    Converts data to laspy format.
    Laspy RGB channels must be of 16-bit format 0-65535.
    These default for the Pointcloud class is 0-255.
    """
    header = lp.LasHeader(point_format=3, version="1.2")

    las = lp.LasData(header)
    las.x = points[:, 0]
    las.y = points[:, 1]
    las.z = points[:, 2]

    if colors is not None:
        point_colors = colors.astype(np.uint16)
        if points.shape[0] != point_colors.shape[0]:
            logging.info("Points and color length are not equal.")
            return
        las.red = point_colors[:, 0]
        las.green = point_colors[:, 1]
        las.blue = point_colors[:, 2]

    if extra_dim is not None:
        if not isinstance(extra_dim, dict):
            logging.info(
                "extra_dim should be a dictionary with the dimname as key, and the value a N1 array or list of values"
            )
            return

        for key, value in extra_dim.items():
            if points.shape[0] != value.shape[0]:
                logging.info("Points and extra dimension length are not equal.")
                return

            if key != "classification":
                las.add_extra_dim(lp.ExtraBytesParams(name=key, type="uint32"))
                setattr(las, key, value)
            else:
                las.classification = value

    return las

### Stap 1: De las data laden

In [98]:
data_folder = "data/utrecht_subset"  # Pad naar de folder met de las/laz files
output_folder = Path(data_folder) / "output"  # Pad naar de folder voor output
os.makedirs(output_folder, exist_ok=True)

In [99]:
lasfiles = get_files_in_directory(data_folder)
print(lasfiles)

lasfile_path = [
    filepath
    for filepath in lasfiles
    if filepath
    == f"data/utrecht_subset/AHN5_C_{int(bbox[0])}_{int(bbox[1])}_{0}_{0}.las"
][0]

lasfile_path

['data/utrecht_subset/AHN5_C_125000_456000_5_5.las', 'data/utrecht_subset/AHN5_C_125000_456000_8_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_9_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_2_1.las', 'data/utrecht_subset/AHN5_C_125000_456000_6_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_4_4.las', 'data/utrecht_subset/AHN5_C_125000_456000_3_9.las', 'data/utrecht_subset/AHN5_C_125000_456000_5_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_0_6.las', 'data/utrecht_subset/AHN5_C_125000_456000_1_9.las', 'data/utrecht_subset/AHN5_C_125000_456000_2_3.las', 'data/utrecht_subset/AHN5_C_125000_456000_7_0.las', 'data/utrecht_subset/AHN5_C_125000_456000_1_5.las', 'data/utrecht_subset/AHN5_C_125000_456000_3_5.las', 'data/utrecht_subset/AHN5_C_125000_456000_4_2.las', 'data/utrecht_subset/AHN5_C_125000_456000_3_7.las', 'data/utrecht_subset/AHN5_C_125000_456000_5_7.las', 'data/utrecht_subset/AHN5_C_125000_456000_8_6.las', 'data/utrecht_subset/AHN5_C_125000_456000_2_6.las', 'data/utrec

'data/utrecht_subset/AHN5_C_125000_456000_0_0.las'

In [100]:
lasfile = lasdata_loader(lasfile_path, single_array=True)

In [104]:
# Zie welke layers er beschikbaar zijn in de BGT data
gpd.list_layers(output_folder / bgt_name)

,name,geometry_type
0,begroeidterreindeel,Polygon
1,onbegroeidterreindeel,Polygon
2,ondersteunendwaterdeel,Polygon
3,ondersteunendwegdeel,Polygon
4,openbareruimte,MultiPolygon
5,pand,MultiPolygon
6,vegetatieobject_punt,Point
7,vegetatieobject_vlak,Polygon
8,waterdeel,Polygon
9,wegdeel,Polygon


In [ ]:
# Laad ook de geodata, kies hier een layer die je interessant lijkt!
geodata = gpd.read_file(output_folder / bgt_name, layer="pand")
geodata.head()

In [91]:
from shapely.geometry import box


def pointcloud_to_bbox_gdf(
    points: np.ndarray, crs: str = "EPSG:28992"
) -> gpd.GeoDataFrame:
    """
    Create a bounding box polygon from a point cloud and return as single-row GeoDataFrame.

    Args:
        points: numpy array with shape (n_points, 2) or (n_points, 3) - X,Y coordinates (Z ignored if present)
        crs: Coordinate reference system for the output GeoDataFrame

    Returns:
        GeoDataFrame with single row containing bounding box polygon and metadata
    """

    # Extract X,Y coordinates (ignore Z if present)
    xy_points = points[:, :2]

    # Calculate bounding box
    min_x, min_y = np.min(xy_points, axis=0)
    max_x, max_y = np.max(xy_points, axis=0)

    # Create bounding box polygon
    bbox_polygon = box(min_x, min_y, max_x, max_y)

    # Create GeoDataFrame with metadata
    gdf = gpd.GeoDataFrame(
        {
            "geometry": [bbox_polygon],
            "min_x": [min_x],
            "min_y": [min_y],
            "max_x": [max_x],
            "max_y": [max_y],
            "width": [max_x - min_x],
            "height": [max_y - min_y],
            "area": [bbox_polygon.area],
            "n_points": [len(points)],
        },
        crs=crs,
    )

    return gdf


def pointcloud_to_bbox_coords(points: np.ndarray) -> list:
    """
    Get bounding box coordinates from point cloud in [min_x, min_y, max_x, max_y] format.

    Args:
        points: numpy array with point coordinates

    Returns:
        List with [min_x, min_y, max_x, max_y]
    """
    xy_points = points[:, :2]
    min_x, min_y = np.min(xy_points, axis=0)
    max_x, max_y = np.max(xy_points, axis=0)

    return [min_x, min_y, max_x, max_y]


def compare_bboxes(
    points: np.ndarray, geodata: gpd.GeoDataFrame, crs: str = "EPSG:28992"
) -> tuple:
    """
    Compare bounding boxes of point cloud and geodata.

    Args:
        points: numpy array with point coordinates
        geodata: GeoDataFrame to compare with
        crs: coordinate reference system

    Returns:
        Tuple of (pointcloud_bbox_gdf, geodata_bbox_gdf, combined_bbox_gdf)
    """
    # Get point cloud bounding box
    pc_bbox = pointcloud_to_bbox_gdf(points, crs)

    # Get geodata bounding box
    geo_bounds = geodata.total_bounds  # [min_x, min_y, max_x, max_y]
    geo_bbox_polygon = box(geo_bounds[0], geo_bounds[1], geo_bounds[2], geo_bounds[3])

    geo_bbox = gpd.GeoDataFrame(
        {
            "geometry": [geo_bbox_polygon],
            "min_x": [geo_bounds[0]],
            "min_y": [geo_bounds[1]],
            "max_x": [geo_bounds[2]],
            "max_y": [geo_bounds[3]],
            "width": [geo_bounds[2] - geo_bounds[0]],
            "height": [geo_bounds[3] - geo_bounds[1]],
            "area": [geo_bbox_polygon.area],
            "n_features": [len(geodata)],
        },
        crs=geodata.crs,
    )

    # Combined bounding box
    combined_min_x = min(pc_bbox.iloc[0]["min_x"], geo_bbox.iloc[0]["min_x"])
    combined_min_y = min(pc_bbox.iloc[0]["min_y"], geo_bbox.iloc[0]["min_y"])
    combined_max_x = max(pc_bbox.iloc[0]["max_x"], geo_bbox.iloc[0]["max_x"])
    combined_max_y = max(pc_bbox.iloc[0]["max_y"], geo_bbox.iloc[0]["max_y"])

    combined_polygon = box(
        combined_min_x, combined_min_y, combined_max_x, combined_max_y
    )
    combined_bbox = gpd.GeoDataFrame(
        {
            "geometry": [combined_polygon],
            "min_x": [combined_min_x],
            "min_y": [combined_min_y],
            "max_x": [combined_max_x],
            "max_y": [combined_max_y],
            "width": [combined_max_x - combined_min_x],
            "height": [combined_max_y - combined_min_y],
            "area": [combined_polygon.area],
        },
        crs=crs,
    )

    return pc_bbox, geo_bbox, combined_bbox

In [92]:
# Test de bounding box functies
print("Creating bounding box from point cloud...")

# Krijg bounding box van de point cloud als GeoDataFrame
pc_bbox_gdf = pointcloud_to_bbox_gdf(lasfile, crs="EPSG:28992")

print("Point cloud bounding box:")
print(pc_bbox_gdf)

# Krijg ook de simpele coördinaten lijst
bbox_coords = pointcloud_to_bbox_coords(lasfile)
print(f"\nBounding box coordinates: {bbox_coords}")

# Vergelijk met BGT data bounding box
if "geodata" in locals():
    print("\nComparing bounding boxes...")
    pc_bbox, geo_bbox, combined_bbox = compare_bboxes(lasfile, geodata, "EPSG:28992")

    print("\nPoint cloud bbox:")
    print(f"  Area: {pc_bbox.iloc[0]['area']:.2f} m²")
    print(
        f"  Dimensions: {pc_bbox.iloc[0]['width']:.1f} x {pc_bbox.iloc[0]['height']:.1f} m"
    )

    print("\nBGT data bbox:")
    print(f"  Area: {geo_bbox.iloc[0]['area']:.2f} m²")
    print(
        f"  Dimensions: {geo_bbox.iloc[0]['width']:.1f} x {geo_bbox.iloc[0]['height']:.1f} m"
    )

    print("\nCombined bbox:")
    print(f"  Area: {combined_bbox.iloc[0]['area']:.2f} m²")
    print(
        f"  Dimensions: {combined_bbox.iloc[0]['width']:.1f} x {combined_bbox.iloc[0]['height']:.1f} m"
    )

Creating bounding box from point cloud...
Point cloud bounding box:
                                            geometry     min_x     min_y  \
0  POLYGON ((125099.999 456000, 125099.999 456099...  125000.0  456000.0   

        max_x       max_y   width  height         area  n_points  
0  125099.999  456099.998  99.999  99.998  9999.700002    218803  

Bounding box coordinates: [np.float64(125000.0), np.float64(456000.0), np.float64(125099.999), np.float64(456099.998)]

Comparing bounding boxes...

Point cloud bbox:
  Area: 9999.70 m²
  Dimensions: 100.0 x 100.0 m

BGT data bbox:
  Area: 0.00 m²
  Dimensions: 0.0 x 0.0 m

Combined bbox:
  Area: 57049335709.84 m²
  Dimensions: 125095.1 x 456047.9 m


In [93]:
pc_bbox_gdf.to_file("bbox0.gpkg", driver="GPKG")

In [47]:
geodata.head()

,bag_pnd,bronhouder,creation_date,eind_registratie,in_onderzoek,in_onderzoek_leeg,lokaal_id,lv_publicatiedatum,plus_status,plus_status_codespace,plus_status_leeg,relatieve_hoogteligging,status,status_codespace,status_leeg,termination_date,termination_date_leeg,tijdstip_registratie,version,geometry
0,0632100000023122,G0632,2015-08-18T22:00:00Z,None,None,geenWaarde,G0632.0019ae63df0c4f71bc3b1a845c912fc7,2016-12-14T16:36:14Z,None,http://www.geostandaarden.nl/imgeo/def/2.1#Voi...,geenWaarde,0,bestaand,http://www.geostandaarden.nl/imgeo/def/2.1#Status,None,None,geenWaarde,2016-11-16T16:29:52Z,301e85af-47e6-2f48-6d84-fd6830e1b749,"MULTIPOLYGON (((4.954 52.093, 4.953 52.093, 4...."
1,0632100000022756,G0632,2015-08-18T22:00:00Z,None,None,geenWaarde,G0632.0097495777ff4605b8d3acfcf3c7fe99,2016-12-14T16:36:14Z,None,http://www.geostandaarden.nl/imgeo/def/2.1#Voi...,geenWaarde,0,bestaand,http://www.geostandaarden.nl/imgeo/def/2.1#Status,None,None,geenWaarde,2016-11-16T16:29:52Z,69bfa306-a0fe-a350-6e93-024fd0eae07f,"MULTIPOLYGON (((4.954 52.093, 4.954 52.093, 4...."
2,0632100000023330,G0632,2015-08-18T22:00:00Z,None,None,geenWaarde,G0632.00ac1d6343b04f45803c6b92cd0d5de0,2016-12-14T16:36:14Z,None,http://www.geostandaarden.nl/imgeo/def/2.1#Voi...,geenWaarde,0,bestaand,http://www.geostandaarden.nl/imgeo/def/2.1#Status,None,None,geenWaarde,2016-11-16T16:29:52Z,c42685c8-dad3-d1ab-8189-3af18e5719a6,"MULTIPOLYGON (((4.958 52.093, 4.958 52.093, 4...."
3,0632100000021863,G0632,2015-08-18T22:00:00Z,2019-01-25T12:31:37Z,None,geenWaarde,G0632.00c6d87629ef42c08b043d731dd6b1bb,2016-12-14T16:36:14Z,None,http://www.geostandaarden.nl/imgeo/def/2.1#Voi...,geenWaarde,0,bestaand,http://www.geostandaarden.nl/imgeo/def/2.1#Status,None,None,geenWaarde,2016-11-16T16:29:52Z,db53aa1b-5534-839b-aae5-01b31e11d7e6,"MULTIPOLYGON (((4.959 52.092, 4.959 52.092, 4...."
4,0632100000023474,G0632,2015-08-18T22:00:00Z,None,None,geenWaarde,G0632.00cc71d01c5f4e4e868362d02ea5397b,2016-12-14T16:36:14Z,None,http://www.geostandaarden.nl/imgeo/def/2.1#Voi...,geenWaarde,0,bestaand,http://www.geostandaarden.nl/imgeo/def/2.1#Status,None,None,geenWaarde,2016-11-16T16:29:52Z,1d88fa2e-b46b-ccf7-b989-4317759e3872,"MULTIPOLYGON (((4.954 52.093, 4.954 52.093, 4...."


In [105]:
def points_in_polygons(points: np.ndarray, gdf: gpd.GeoDataFrame) -> np.ndarray:
    """
    Find which polygon each point falls into.

    Args:
        points: numpy array with shape (n_points, 2) or (n_points, 3) - X,Y coordinates (Z ignored if present)
        gdf: GeoDataFrame with polygon geometries

    Returns:
        numpy array with shape (n_points,) containing the index of the polygon each point falls into.
        Returns -1 if point doesn't fall in any polygon.
    """
    from shapely.geometry import Point
    from shapely.strtree import STRtree

    # Extract X,Y coordinates (ignore Z if present)
    xy_points = points[:, :2]

    # Create Point geometries from coordinates
    point_geoms = [Point(x, y) for x, y in xy_points]

    # Create spatial index for fast lookup
    tree = STRtree(gdf.geometry.values)

    # Initialize result array with -1 (no polygon found)
    result = np.full(len(points), -1, dtype=int)

    # For each point, find which polygon it falls into
    for i, point_geom in enumerate(point_geoms):
        # Query spatial index for potential matches
        possible_matches_idx = list(tree.query(point_geom))

        # Check actual containment for potential matches
        for idx in possible_matches_idx:
            if gdf.geometry.iloc[idx].contains(point_geom):
                result[i] = idx
                break  # Found the polygon, move to next point

    return result


def points_in_polygons_with_info(
    points: np.ndarray, gdf: gpd.GeoDataFrame, info_columns: list = None
) -> np.ndarray:
    """
    Enhanced version that returns both polygon indices and additional information.

    Args:
        points: numpy array with shape (n_points, 2) or (n_points, 3) - X,Y coordinates
        gdf: GeoDataFrame with polygon geometries
        info_columns: list of column names to extract info from (e.g., ['bgt_functie', 'plus_type'])

    Returns:
        numpy array with polygon indices and a list of dictionaries with additional info
    """
    indices = points_in_polygons(points, gdf)

    if info_columns is None:
        return indices, None

    # Extract additional information for each point
    point_info = []
    for idx in indices:
        if idx == -1:
            # Point not in any polygon
            info = {col: None for col in info_columns}
        else:
            # Extract info from the polygon
            info = {
                col: gdf.iloc[idx][col] for col in info_columns if col in gdf.columns
            }
        point_info.append(info)

    return indices, point_info


# Example usage function
def classify_points_by_polygons(
    points: np.ndarray,
    gdf: gpd.GeoDataFrame,
    classification_column: str = "bgt_functie",
) -> np.ndarray:
    """
    Classify points based on the polygon they fall into.

    Args:
        points: numpy array with point coordinates
        gdf: GeoDataFrame with polygons
        classification_column: column name to use for classification

    Returns:
        numpy array with original points plus classification as 4th column
    """
    # Get polygon indices
    polygon_indices = points_in_polygons(points, gdf)

    # Create result array with 4 columns (X, Y, Z, classification)
    if points.shape[1] == 2:
        # Add Z=0 if only X,Y provided
        result = np.zeros((len(points), 4))
        result[:, :2] = points
    else:
        result = np.zeros((len(points), 4))
        result[:, :3] = points[:, :3]

    # Add classification based on polygon
    for i, poly_idx in enumerate(polygon_indices):
        if poly_idx != -1 and classification_column in gdf.columns:
            # Use the polygon index as classification (you can modify this logic)
            result[i, 3] = poly_idx
        else:
            result[i, 3] = -1  # No polygon found

    return result

In [106]:
# Test de point-in-polygon functie met je data
print("Testing point-in-polygon classification...")

# Neem een subset van punten voor snellere testing
test_points = lasfile  # Eerste 1000 punten
print(f"Testing with {len(test_points)} points")

# Vind voor elk punt in welk BGT polygoon het valt
polygon_indices = points_in_polygons(test_points, geodata)

# Toon resultaten
print(f"Points analyzed: {len(polygon_indices)}")
print(f"Points in polygons: {np.sum(polygon_indices != -1)}")
print(f"Points outside polygons: {np.sum(polygon_indices == -1)}")

# Toon welke polygon types we hebben gevonden
unique_indices = np.unique(polygon_indices[polygon_indices != -1])
print(f"Found {len(unique_indices)} different polygons")

# Als je BGT functie informatie wilt zien
if "bgt_functie" in geodata.columns:
    print("\nBGT functies gevonden:")
    for idx in unique_indices[:10]:  # Eerste 10
        functie = geodata.iloc[idx]["bgt_functie"]
        count = np.sum(polygon_indices == idx)
        print(f"  Polygon {idx}: {functie} ({count} punten)")

Testing point-in-polygon classification...
Testing with 218803 points
Points analyzed: 218803
Points in polygons: 15073
Points outside polygons: 203730
Found 9 different polygons


De lasfile is nu geladen als een numpy array. Een matrix met de kolommen X,Y,Z en elke rij 1 punt.
je kan dit array ook filteren op index. Zie: https://numpy.org/doc/stable/user/basics.indexing.html
Python begint met tellen bij 0. Het derde punt in de lasfile krijg je daarom door:
```
lasfile[2, :]
```

Wil je alle Y-coordinaten? dan gebruik je:
```
lasfile[:, 1]
```

Beantwoord de volgende vragen:

In [8]:
# Vraag 1: hoeveel punten zitten er in deze lasfile? (antwoord: 1 regel code)
len(lasfile)

363668

In [9]:
# vraag 2: Hoeveel dimensies heeft deze lasfile? Deze vraag kan je beantwoorden zonder code.
lasfile.shape

(363668, 3)

In [10]:
# Vraag 3: wat is de x,y,z coordinaten van het 10e punt in deze lasfile? (antwoord: 1 regel code)
lasfile[9, :]

array([ 1.25502578e+05,  4.56597670e+05, -1.25000000e-01])

Daarnaast kan je ook dingen als min, max en gemiddelde berekenen van een subselectie. Probeer zelf erachter te komen hoe je dit doet! Beantwoord daarbij de volgende vragen:

In [11]:
# Vraag 4: Wat is de gemiddelde hoogte (z) van alle punten in deze lasfile? (antwoord: 1 regel code)
lasfile[:, 2].mean()

np.float64(0.24662079149114033)

In [12]:
# Vraag 5: Wat is de min en max X-waarde van alle punten in deze lasfile? (antwoord: 1 regel code)
lasfile[:, 0].min(), lasfile[:, 0].max()

(np.float64(125500.0), np.float64(125599.999))

In [13]:
# Vraag 6: Wat is de min en max Y-waarde van alle punten in deze lasfile? (antwoord: 1 regel code)
lasfile[:, 1].min(), lasfile[:, 1].max()

(np.float64(456500.0), np.float64(456599.999))

In [14]:
# Waarom had je deze X en Y waarden al kunnen verwachten zonder code?

Nu willen we de puntenwolk gaan "ophakken" in nog kleinere subsets. Vervolgens classificeren we alle punten die in een subset vallen ans behorend bij die subset.
Laten we de puntenwolk ophakken in een 3x3 matrix.

In [15]:
# Maak een kopie van de originele array en voeg een 4e kolom toe. Deze vullen we met 0-en.
lasfile_with_grid = np.zeros((len(lasfile), 4))
lasfile_with_grid[:, :3] = lasfile  # Kopieer X, Y, Z coordinaten

In [16]:
# Bereken de bounding box van de point cloud
x_min, x_max = lasfile_with_grid[:, 0].min(), lasfile_with_grid[:, 0].max()
y_min, y_max = lasfile_with_grid[:, 1].min(), lasfile_with_grid[:, 1].max()
print(f"X range: {x_min:.2f} to {x_max:.2f}")
print(f"Y range: {y_min:.2f} to {y_max:.2f}")

X range: 125500.00 to 125600.00
Y range: 456500.00 to 456600.00


In [17]:
# Bereken de cell grootte voor de 3x3 matrix
grid_size = 3
x_step = (x_max - x_min) / grid_size
y_step = (y_max - y_min) / grid_size

print(f"Cell grootte: {x_step:.2f} x {y_step:.2f}")

Cell grootte: 33.33 x 33.33


In [18]:
# Create a dictionary to store points for each grid cell
grid_cells = {}

# Per punt de grid cell assignment:
for i, point in enumerate(lasfile_with_grid):
    x, y, z = point[:3]

    # Bereken welk grid cell het punt in valt
    grid_x = int((x - x_min) / x_step)
    grid_y = int((y - y_min) / y_step)

    # Edge cases die precies op de grens vallen
    if grid_x == grid_size:
        grid_x = grid_size - 1
    if grid_y == grid_size:
        grid_y = grid_size - 1

    # Converteer grid coordinaten naar een enkele classificatie waarde (0-8)
    # Grid layout: 0 1 2
    #              3 4 5
    #              6 7 8
    grid_classification = grid_y * grid_size + grid_x

    # Voeg de grid classificatie toe als 4e dimensie
    lasfile_with_grid[i, 3] = grid_classification

    # Voeg het punt toe aan de dictionary
    if grid_classification not in grid_cells:
        grid_cells[grid_classification] = []
    grid_cells[grid_classification].append(point)

# Print statistics for each grid cell
print("Grid cell statistics:")
for cell_key in grid_cells.keys():
    if cell_key in grid_cells:
        point_count = len(grid_cells[cell_key])
        print(f"Cell {cell_key}: {point_count} points")
    else:
        print(f"Cell ({cell_key}): 0 points")

print(
    f"\nTotal points distributed: {sum(len(points) for points in grid_cells.values())}"
)
print(f"Original point count: {len(lasfile)}")

Grid cell statistics:
Cell 6: 27118 points
Cell 7: 38461 points
Cell 3: 28551 points
Cell 0: 39287 points
Cell 4: 47537 points
Cell 1: 51500 points
Cell 5: 42380 points
Cell 8: 47320 points
Cell 2: 41514 points

Total points distributed: 363668
Original point count: 363668


In [19]:
lasfile_with_grid[:4]

array([[ 1.25501602e+05,  4.56593789e+05, -3.90000000e-02,
         6.00000000e+00],
       [ 1.25501393e+05,  4.56594134e+05,  1.11000000e-01,
         6.00000000e+00],
       [ 1.25500738e+05,  4.56596006e+05, -7.90000000e-02,
         6.00000000e+00],
       [ 1.25501951e+05,  4.56594276e+05,  1.73800000e+00,
         6.00000000e+00]])

In [20]:
# Creëer een nieuwe numpy array met 4 dimensies: X, Y, Z, en grid classificatie
# De grid classificatie is een getal van 0-8 (voor een 3x3 grid)

# Maak een kopie van de originele array en voeg een 4e kolom toe
lasfile_with_grid = np.zeros((len(lasfile), 4))
lasfile_with_grid[:, :3] = lasfile  # Kopieer X, Y, Z coordinaten

# Voeg grid classificaties toe voor elk punt
for i, point in enumerate(lasfile):
    x, y, z = point

    # Bereken welke grid cell dit punt hoort
    grid_x = int((x - x_min) / x_step)
    grid_y = int((y - y_min) / y_step)

    # Handle edge case where point is exactly on the boundary
    if grid_x == grid_size:
        grid_x = grid_size - 1
    if grid_y == grid_size:
        grid_y = grid_size - 1

    # Converteer grid coordinaten naar een enkele classificatie waarde (0-8)
    # Grid layout: 0 1 2
    #              3 4 5
    #              6 7 8
    grid_classification = grid_y * grid_size + grid_x

    # Voeg de grid classificatie toe als 4e dimensie
    lasfile_with_grid[i, 3] = grid_classification

print("Nieuwe array shape:", lasfile_with_grid.shape)
print("Eerste 10 punten met grid classificatie:")
print("X\t\tY\t\tZ\t\tGrid")
for i in range(10):
    x, y, z, grid_class = lasfile_with_grid[i]
    print(f"{x:.2f}\t\t{y:.2f}\t\t{z:.2f}\t\t{int(grid_class)}")

# Toon hoeveel punten er in elke grid classificatie zitten
print("\nPunten per grid classificatie:")
unique_grids, counts = np.unique(lasfile_with_grid[:, 3], return_counts=True)
for grid_id, count in zip(unique_grids, counts):
    grid_x = int(grid_id) % grid_size
    grid_y = int(grid_id) // grid_size
    print(f"Grid {int(grid_id)} (cel {grid_x},{grid_y}): {count} punten")

Nieuwe array shape: (363668, 4)
Eerste 10 punten met grid classificatie:
X		Y		Z		Grid
125501.60		456593.79		-0.04		6
125501.39		456594.13		0.11		6
125500.74		456596.01		-0.08		6
125501.95		456594.28		1.74		6
125502.26		456595.29		-0.13		6
125501.05		456598.28		-0.07		6
125500.90		456598.65		-0.09		6
125501.66		456599.10		0.01		6
125501.38		456599.88		-0.03		6
125502.58		456597.67		-0.12		6

Punten per grid classificatie:
Grid 0 (cel 0,0): 39287 punten
Grid 1 (cel 1,0): 51500 punten
Grid 2 (cel 2,0): 41514 punten
Grid 3 (cel 0,1): 28551 punten
Grid 4 (cel 1,1): 47537 punten
Grid 5 (cel 2,1): 42380 punten
Grid 6 (cel 0,2): 27118 punten
Grid 7 (cel 1,2): 38461 punten
Grid 8 (cel 2,2): 47320 punten


Jammer genoeg biedt laspy geen mogelijkheid om pointclouds te visualiseren. Gelukkig kunnen we hier een ander package voor gebruiken die veel wordt gebruikt voor pointclouds en 3D modellen: open3D!

In [ ]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(lasfile_with_grid[:, :3])
o3d.visualization.draw_geometries([geom])

Als we de classificatie buiten python willen bekijken, kunnen we deze het beste opslaan als nieuwe las file. We moeten het numpy array weer terugveranderen in een las dataset zodat laspy deze kan opslaan. Door het numpy array om te zetten in lasdata zorgen we dat deze voldoet aan de specificaties voor lasdata (zoals een header met de juiste informatie).

In [46]:
las_classified = to_las(
    points=lasfile_with_grid[:, 0:3],
    colors=None,
    extra_dim={"grid_classification": lasfile_with_grid[:, 3].astype("uint32")},
)

In [59]:
las_path = output_folder / Path(Path(lasfile_path).stem + "_grid_classification.las")
las_classified.write(las_path)